# Notebook for postprocessing TLS2Tree results

The goal of this notebook is to merge and substract found instance points from the input cloud to be able to calcualte accuracy metrics. Use venv from home for this.

In [1]:
import open3d as o3d
import numpy as np
import glob

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Paths to source PLY files (to be merged)
inst_files = glob.glob('/home/hannah.greven/individual-tree-segmentation/data/quick-test-out/tls2trees/clouds/*.leafon.ply')
inst_list = [o3d.io.read_point_cloud(f) for f in inst_files]

In [3]:
# Merge all PLY files into a single cloud
all_inst_points = np.vstack([np.asarray(p.points) for p in inst_list])
#inst_merged_cloud = o3d.geometry.PointCloud()
#inst_merged_cloud.points = o3d.utility.Vector3dVector(all_inst_points)

In [4]:
# Paths to source PLY files (to be merged)
tile_files = glob.glob('/home/hannah.greven/individual-tree-segmentation/data/quick-test-out/tls2trees/extraction/downsample/*.ply')
tile_list = [o3d.io.read_point_cloud(f) for f in tile_files]

In [5]:
# Merge all PLY files into a single cloud
ref_points = np.vstack([np.asarray(p.points) for p in tile_list])
#ref_cloud = o3d.geometry.PointCloud()
#ref_cloud.points = o3d.utility.Vector3dVector(all_inst_points)

In [7]:
def get_xyz_mat(points):
    xyz = np.trunc(points[:, 0] * 1e4) / 1e4, np.trunc(points[:, 1] * 1e4) / 1e4, np.trunc(points[:, 2] * 1e4) / 1e4
    return np.stack(xyz, axis=1)

In [8]:
def get_xyz_arr(points):
    xyz = np.trunc(points[0] * 1e4) / 1e4, np.trunc(points[1] * 1e4) / 1e4, np.trunc(points[2] * 1e4) / 1e4
    return xyz

In [9]:
# Find unique points in reference not present in merged cloud
merged_set = set(map(tuple, get_xyz_mat(all_inst_points)))
keep_mask = [tuple(get_xyz_arr(p)) not in merged_set for p in ref_points]
residual_points = ref_points[keep_mask]

In [10]:
all_inst_points.__len__()

3229750

In [11]:
residual_points.__len__()

9233764

In [12]:
ref_points.__len__()

12463514

In [13]:
ref_points.__len__() == all_inst_points.__len__() + residual_points.__len__()

True

In [14]:
# Create new point cloud for residual points
residual_cloud = o3d.geometry.PointCloud()
residual_cloud.points = o3d.utility.Vector3dVector(residual_points)

# Save result to disk
o3d.io.write_point_cloud('data/tmp/tls2trees_residual.ply', residual_cloud)

True

In [69]:
test_pcd = o3d.io.read_point_cloud('/home/hannah.greven/individual-tree-segmentation/data/quick-test/test12.ply')
test_points = np.asarray(test_pcd.points)

In [71]:
test_points.__len__() - ref_points.__len__()

1010534

In [73]:
# Find unique points in reference not present in merged cloud
merged_set = set(map(tuple, get_xyz_mat(ref_points)))
keep_mask = [tuple(get_xyz_arr(p)) in merged_set for p in test_points]
downsample_test_points = test_points[keep_mask]

In [74]:
downsample_test_points.__len__()

21